# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib.patches as mpatches
# import math
import os,re
# from functools import partial
pd.set_option('display.max_columns', None)
import datetime
from matplotlib import rc
rc('text',usetex=True)
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
rc('text.latex', preamble='\\usepackage{color} \\usepackage{amsmath} \\usepackage{amssymb}')

# Path Setting
**NOTE** Comment uncomment LINE No 4,5 to change dataset path

In [ ]:
path_curr = os.getcwd()
path_notebook = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

#path_dataset = os.path.abspath(os.path.join(path_notebook,'dataset/CelebA/CelebAMask-HQ'))
path_dataset = 'D:\\DS\\CelebAMask-HQ\\'

path_main = path_curr

results_path_main   = os.path.join(path_curr,               'results')
# path_csv            = os.path.join(results_path_main,           'csv')
path_csv    = '../results_logits_with_auc_clipped/'
utils_path          = os.path.join(path_notebook,             'utils')

print('path_dataset: \t',        path_dataset)
print('codes path: \t',             path_curr)
print('csv path: \t',                path_csv)
print('utils path: \t',            utils_path)

In [ ]:
if os.path.exists(results_path_main):
    results_boxplots     = os.path.join(results_path_main,  'boxplots')
    os.makedirs(results_boxplots,              exist_ok=True)

# XAI Methods

In [ ]:
method_names = ['BPT-100',
 'BPT-500',
 'BPT-1000',
 'AA-100',
 'AA-500',
 'AA-1000',
 'LIME-100',
 'LIME-500',
 'LIME-1000',
 'LRP',
 'GradCAM',
 'aIDG',
 'aGradExpl',
 'GradShapE']

# Setting for CSV File Name

In [ ]:
# background_type        = 'blurred'     # black , white , gray , noise , blurred , full
# background_type        = 'black'     # black , white , gray , noise , blurred , full
# background_type        = 'white'     # black , white , gray , noise , blurred , full
# background_type        = 'noise'     # black , white , gray , noise , blurred , full
background_type        = 'gray'     # black , white , gray , noise , blurred , full
# background_type        = 'full'     # black , white , gray , noise , blurred , full

In [ ]:
exp_type = 'E5'

In [ ]:
results_path  = os.path.join(results_path_main,('bg_'+background_type))

# Load CSV

In [ ]:
# list_eyeglasses_true - list_brownhairs
subfile_count = 1000

# files_sub,filter_type  = list_brownhairs[:subfile_count],'brownhairs'
iterations = subfile_count #len(files_sub)

In [ ]:
csv_filename = f'{path_csv}/csv_exp_{exp_type}_IoU_face_{iterations}_{len(method_names)}_{background_type}_brownhairs.csv'
print(csv_filename,os.path.exists(csv_filename))

In [ ]:
df_main= pd.read_csv(csv_filename)
df_main

In [ ]:
df_main.loc[(df_main.f_S < df_main.f_0), 'au_IoU'] = np.nan
df_main.loc[(df_main.f_S < df_main.f_0), 'max_IoU'] = np.nan

In [ ]:
df_main.head()

In [ ]:
def inspect_df(df):
    computed_methods = df.method.unique()
    num_methods = len(computed_methods)
    num_images  = len(np.unique(df.image))
    return computed_methods,num_methods,num_images

In [ ]:
computed_methods,num_methods,num_images = inspect_df(df_main)
print(num_methods,computed_methods)
print('-'*120)
print(f'Total Images: {num_images}')

# REMOVE LRP

In [ ]:
# df_main = df_main[df_main.method != 'LRP']

In [ ]:
computed_methods,num_methods,num_images = inspect_df(df_main)
print(num_methods,computed_methods)
print('-'*120)
print(f'Total Images: {num_images}')

In [ ]:
# gt1 = df_main[df_main.aucI_r>10].image

# print(len(gt1.unique()))

# print(gt1.unique())
# prob_images = gt1.unique()


# df_main[df_main.image.isin(prob_images)].sort_values('aucI_r')

# Set Params for Plots

In [ ]:
name_remap = {
    'Partition-100': 'PE-100',
    'Partition-500': 'PE-500',
    'Partition-1000': 'PE-1000',
    'BPT-100': '\\bf BPT-100',
    'BPT-500': '\\bf BPT-500',
    'BPT-1000': '\\bf BPT-1000',
    'aIDG': '$|$IDG$|$',
    'aGradExpl': '$|$GradExpl$|$',
}
method_remap = {
    'max_IoU': '$max$-$IoU$',
    'au_IoU': '$AU$-$IoU$',
    'aucI_pred': '$\\mathit{AUC}^+$',
    'aucD_pred': '$\\mathit{AUC}^-$',
    'aucI_mse': '$\\mathit{MSE}^+$',
    'aucD_mse': '$\\mathit{MSE}^-$',
    'time_exp': 'log(time)'
}
method_colors = {
    'BPT-100':         '#f2a7c0',
    'BPT-500':         '#db587f',
    'BPT-1000':        '#9d2f4d',
    'AA-100':   '#4ed1f9',
    'AA-500':   '#00b1f2',
    'AA-1000':  '#008fe8',
    'AA-5000':  '#006bd7',
    'AA-10000':  '#3245bd',
    'LIME-100':         '#c7e883',
    'LIME-500':        '#8fc543',
    'LIME-1000':        '#66912d',
    'LRP':        '#008080',
    'GradCAM':         'xkcd:camel',
    'aIDG':            '#fcd459',
    'aGradExpl':       '#dd7c2b',
    'GradShapE':       '#800000',
}
method_symbol = {
    'BPT-100'   :       '$\\clubsuit$1',
    'BPT-500'   :       '$\\clubsuit$2',
    'BPT-1000'  :       '$\\clubsuit$3',
    'AA-100'    :       '$\\boxplus$1',
    'AA-500'    :       '$\\boxplus$2',
    'AA-1000'   :       '$\\boxplus$3',
    'AA-5000'   :       '$\\boxplus$4',
    'AA-10000'  :       '$\\boxplus$5',
    'LIME-100'   :       '\\textbf{0}1',
    'LIME-500'  :      '\\textbf{0}2',
    'LIME-1000'  :      '\\textbf{0}3',
    'LRP'       :      '\\textbf{$\\Pi$}',
    'GradCAM'   :      '$\\bigtriangledown$',
    'aIDG'      :      '$\\divideontimes$',
    'aGradExpl' :     '$\\bigtriangleup$',
     'GradShapE':       '$\\square$',
}

method_to_shortname = {}
for i,m in enumerate(method_colors.keys()):
    method_to_shortname[m] = f'{method_symbol[m]}'

# Draw Box Plot

In [ ]:
method_remap = {
    'max_IoU': '$max$-$IoU$',
    'au_IoU': '$AU$-$IoU$',
    'aucI_pred': '$\\mathit{AUC}^+$',
    'aucD_pred': '$\\mathit{AUC}^-$',
    'aucI_adj': '$\\mathit{AUC}^+-Adju$',
    'aucD_adj': '$\\mathit{AUC}^--Adju$',
    
    'aucI_r': '$\\mathit{AUC}^+-R$',
    'aucD_r': '$\\mathit{AUC}^--R$',

    'aucI_adj_r': '$\\mathit{AUC}^+-Adju-R$',
    'aucD_adj_r': '$\\mathit{AUC}^--Adju-R$',


    'aucI_clip': '$\\mathit{AUC}^+-Cl$',
    'aucI_clipr': '$\\mathit{AUC}^+-Cl-R$',

    'aucD_clip': '$\\mathit{AUC}^--Cl$',
    'aucD_clipr': '$\\mathit{AUC}^--Cl-R$',

    'time_exp': 'log(time)'
}

In [ ]:
explained_class_final = 'Brown_hairs and Eyeglasses'
first_400_images = df_main.image.unique()[:400]

df_main_400 = df_main[df_main.image.isin(first_400_images)].reset_index(drop=True)
df_main_400

## SET DATAFRAME FOR PLOTTING EITHER ON FULL IMAGES 449 or 400

In [ ]:
data_frame_for_plotting = df_main_400


In [ ]:
def box_plot(ax, data, label, position, edge_color, line_width, fill_color):
    bp = ax.boxplot(data,
                    # labels=[label],
                    tick_labels=[label],
                    positions=[position], vert=False, 
                    widths=0.5, whis=5, patch_artist=True, flierprops={'markersize': 1, 'markerfacecolor':'magenta', 'markeredgecolor':'magenta'}) # 
    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color=edge_color, linewidth=line_width)

    for patch in bp['boxes']:
        patch.set(facecolor=fill_color)       
        
    return bp


# auc_methods      = ['aucI_pred', 'aucD_pred', 'aucI_adj', 'aucD_adj', 'aucI_r','aucD_r',\
#                     'aucI_adj_r','aucD_adj_r', 'max_IoU', 'au_IoU', 'time_exp']

auc_methods      = ['aucI_pred','aucI_clipr' ,'aucI_clip', 'aucI_adj' , 'aucI_r', 'aucI_adj_r', 'max_IoU','time_exp',\
                    'aucD_pred','aucD_clipr','aucD_clip', 'aucD_adj' , 'aucD_r', 'aucD_adj_r','au_IoU',\
                    ]

auc_method_ascending = ['aucI_pred','aucI_clipr','aucI_clip','aucI_adj','aucI_r', 'aucI_adj_r','max_IoU', 'au_IoU' ]

total_images = len(data_frame_for_plotting.image.unique())
total_methods = len(data_frame_for_plotting.method.unique())
ttl_cols = len(auc_methods) if len(auc_methods)%2==0 else len(auc_methods)+1 


print(f'Explained Class: {explained_class_final}')
print(f'Total Images: {total_images}')
print(f'XAI Methods: {total_methods}')
print('Background type:\t',background_type)
print('Total Time: \t\t',str(datetime.timedelta(seconds=np.sum(data_frame_for_plotting.time_total))))

fig, axes = plt.subplots(2, int(ttl_cols/2), figsize=(15, 7))  # (rows, cols)
for j, score in enumerate(auc_methods):
    row, col = divmod(j, int(ttl_cols/2))
    ax = axes[row, col]
    # data_frame_for_plotting = data_frame_for_plotting[~data_frame_for_plotting[score].isna()] 
    # df = df
    ascending = score in auc_method_ascending
    means = data_frame_for_plotting.groupby('method')[[score]].median() \
              .sort_values(score, ascending=ascending).index
    log_scale = (score=='time_exp')
    ax.set_xscale('log' if log_scale else 'linear')
    for i, method in enumerate(means):
        text_yincrement = -0.3
        values = data_frame_for_plotting[ data_frame_for_plotting.method==method ][score]

        values = values[~values.isna()]
        name = name_remap[method] if method in name_remap else method
        color = method_colors[method]
        q75 = np.quantile(values, q=0.75)
        q100 = np.quantile(values, q=1.0)
        if not log_scale and 'IoU' not in score and q100>1.0:
            q100, text_yincrement = 0.82, 0.15
        val_arr = [[v for v in values]]
        box_plot(ax, val_arr, '', i, 
                 'blue' if method.startswith('BPT') else 'black', 
                 1.5 if method.startswith('BPT') else 0.5,
                 color)
        # print(method,q100,score,np.max(values))
        if score in ['aucI_pred', 'aucD_pred']:
            ax.text(min(q100*1.3, 30) if log_scale else q75+0.5, i+text_yincrement, method_to_shortname[method])
        elif score in ['aucI_mse', 'aucD_mse']:
            ax.text(min(q100*1.3, 30) if log_scale else q100+60, i+text_yincrement, method_to_shortname[method])
        else:
            ax.text(min(q100*1.3, 30) if log_scale else q100+0.05, i+text_yincrement, method_to_shortname[method])

    #################################################
    values = data_frame_for_plotting[score]
    ##########################################
    if score=='aucD_pred': min_x,max_x= 0,1 
    
    if score=='aucI_r' : min_x,max_x= 0,1
    if score=='aucD_mse' : min_x,max_x= 0,1 
    
    if score=='aucI_mse' : min_x,max_x= -np.quantile(values,q=0.01),400
    if score=='aucI_mse' : min_x,max_x= -np.quantile(values,q=0.01),400

    if score=='max_IoU'  : min_x,max_x= -np.quantile(values,q=0.01),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*20)
    if score=='au_IoU'   : min_x,max_x= -np.quantile(values,q=0.01),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*30)
    ##########################################

    if score=='time_exp' :
        min_x, max_x = 0.011, 99
        for m in [0.01, 0.1, 1, 10]:
            for k in range(1,10):
                ax.axvline(m*k, lw=0.5, color='grey', alpha=1/k)
        ax.set_xticks([0.1, 1, 10])
        ax.set_xticklabels(['0.1', '1', '10'])
    if score=='aucI_r' : 
        min_x,max_x= 0,5
        ax.set_xlim(min_x, max_x)
    # ax.set_xlim(min_x, max_x)
    ax.set_title(f"{method_remap[score]}", fontsize=15)

    ax.set_ylim(-0.7, i+0.8)
    ax.set_yticks([])

plt.subplots_adjust(wspace=0.10)
filename_box_plot = f'{results_boxplots}/results_table_IoU_{len(np.unique(data_frame_for_plotting.image))}_{len(method_names)}_{background_type}'

plt.savefig(f'{filename_box_plot}.pdf', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
plt.savefig(f'{filename_box_plot}.svg', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
# plt.savefig(f'{filename_box_plot}.png', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)


################################
from matplotlib.patches import Patch

last_ax = axes[-1, -1]

lm = list(df_main.method.unique())

legend_handles = []
for m in lm:
    shortname = method_to_shortname.get(m, m)
    fullname = name_remap.get(m, m)
    label = f'{shortname}: {fullname}'
    patch = Patch(
        facecolor=method_colors[m],
        edgecolor='blue' if m.startswith('BPT') else 'black',
        linewidth=1.5 if m.startswith('BPT') else 0.5,
        label=label
    )
    legend_handles.append(patch)
for ti, tl in zip(('Total Images','XAI Methods'),(total_images,total_methods)):
    summary_patch = Patch(facecolor='white', edgecolor='white', label=[ti,tl])
    legend_handles.append(summary_patch)

last_ax.legend(
    handles=legend_handles,
    ncol=1,
    loc='center left',
    fontsize=7,
    frameon=False
)
plt.suptitle(f'{exp_type}_{explained_class_final} Total Images : {total_images}')
plt.suptitle(f'[Experiment:{exp_type}_CelebA_Yolo] --- [Explained Class: {explained_class_final}]\n[Total Images : {total_images}] --- [total_methods : {total_methods}]\
             --- [Total Time: {str(datetime.timedelta(seconds=np.sum(df_main.time_total)))}]', fontsize=16)

plt.show()
print('file saved at :', filename_box_plot)

In [ ]:
df_main.intrested_class_lbl.unique()

## FOR PAPER (SUPPLIMENTARY) ONLY

In [ ]:
method_to_shortname

In [ ]:
method_remap = {
    'max_IoU': '$max$-$IoU$',
    'au_IoU': '$AU$-$IoU$',

    'aucI_clipr': '$\\mathit{AUC}^+$',
    'aucD_clipr': '$\\mathit{AUC}^-$',

    'time_exp': 'log(time)'
}

arrow_up = '$\\Uparrow$'
arrow_down = '$\\Downarrow$'
arrow = {

    'max_IoU': arrow_up,
    'au_IoU': arrow_up,
    'aucI_clipr': arrow_up,
    
    'aucD_clipr': arrow_down,
    'time_exp': arrow_down,
}

In [ ]:
def box_plot(ax, data, label, position, edge_color, line_width, fill_color):
    bp = ax.boxplot(data,
                    # labels=[label],
                    tick_labels=[label],
                    positions=[position], vert=False, 
                    widths=0.5, whis=5, patch_artist=True, flierprops={'markersize': 1, 'markerfacecolor':'magenta', 'markeredgecolor':'magenta'}) # 
    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color=edge_color, linewidth=line_width)

    for patch in bp['boxes']:
        patch.set(facecolor=fill_color)       
        
    return bp


auc_methods      = [
                    # 'aucI_pred','aucD_pred',\
                    'aucI_clipr' ,'aucD_clipr', \
                    'max_IoU','au_IoU',\
                    'time_exp']

auc_method_ascending = ['aucI_clipr','aucI_clip','aucI_adj','aucI_r', 'aucI_adj_r','max_IoU', 'au_IoU' ]

total_images = len(data_frame_for_plotting.image.unique())
total_methods = len(data_frame_for_plotting.method.unique())
ttl_cols = len(auc_methods) if len(auc_methods)%2==0 else len(auc_methods)+1 


print(f'Explained Class: {explained_class_final}')
print(f'Total Images: {total_images}')
print(f'XAI Methods: {total_methods}')
print('Background type:\t',background_type)
print('Total Time: \t\t',str(datetime.timedelta(seconds=np.sum(data_frame_for_plotting.time_total))))

fig, axes = plt.subplots(1, len(auc_methods), figsize=(12, 3))  # (rows, cols)
for j, score in enumerate(auc_methods):
    # row, col = divmod(j, int(ttl_cols/2))
    ax = axes[j]
    ascending = score in auc_method_ascending
    means = data_frame_for_plotting.groupby('method')[[score]].median() \
              .sort_values(score, ascending=ascending).index
    
    # print('means', means)
    log_scale = (score=='time_exp')
    ax.set_xscale('log' if log_scale else 'linear')
    for i, method in enumerate(means):
        text_yincrement = -0.3
        values = data_frame_for_plotting[ data_frame_for_plotting.method==method ][score]

        values = values[~values.isna()]
        # values = 0 if score=='time_exp'
        name = name_remap[method] if method in name_remap else method
        color = method_colors[method]
        q75 = np.quantile(values, q=0.75)
        q100 = np.quantile(values, q=1.0)
        if not log_scale and 'IoU' not in score and q100>1.0:
            q100, text_yincrement = 0.82, 0.15
        val_arr = [[v for v in values]]
        box_plot(ax, val_arr, '', i, 
                 'blue' if method.startswith('BPT') else 'black', 
                 1.5 if method.startswith('BPT') else 0.5,
                 color)
        # print(method,q100,score,np.max(values))
        if score in ['aucI_pred', 'aucD_pred']:
            ax.text(min(q100*1.3, 30) if log_scale else q75+0.5, i+text_yincrement, method_to_shortname[method])
        elif score in ['aucI_mse', 'aucD_mse']:
            ax.text(min(q100*1.3, 30) if log_scale else q100+60, i+text_yincrement, method_to_shortname[method])
        elif score =='time_exp':
            ax.text(min(q100*1.3, 30) if log_scale else q100+0.5, i+text_yincrement, method_to_shortname[method])
        else:
            ax.text(min(q100*1.3, 30) if log_scale else q100+0.05, i+text_yincrement, method_to_shortname[method])
    values = data_frame_for_plotting[score]
    # print('values len', len(values))

    ##########################################
    if score=='aucI_clipr' : min_x,max_x= -0.05,1.2
    if score=='aucD_clipr' : min_x,max_x= -0.05,1.2

    if score=='max_IoU'  : min_x,max_x=-0.05,1.1 
    if score=='au_IoU'   : min_x,max_x= -0.05, 0.7  
    ##########################################
    if score=='time_exp' :
        min_x, max_x = 0.001, 99
        for m in [0.001, 0.1, 1, 10]:
            for k in range(1,10):
                ax.axvline(m*k, lw=0.5, color='grey', alpha=1/k)
        ax.set_xticks([0.001, 1, 10])
        ax.set_xticklabels(['0.001', '1', '10'])
    if score=='aucI_r' : 
        min_x,max_x= 0,5
        ax.set_xlim(min_x, max_x)
    ax.set_xlim(min_x, max_x)
    ax.set_title(f"{arrow[score]} {method_remap[score]}", fontsize=15)
    ax.set_ylim(-0.7, i+0.8)
    ax.set_yticks([])

plt.subplots_adjust(wspace=0.10)
filename_box_plot = f'{results_boxplots}/results_CelebA_table_IoU_{len(np.unique(data_frame_for_plotting.image))}_{len(method_names)}_{background_type}'

plt.savefig(f'{filename_box_plot}.pdf', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
plt.savefig(f'{filename_box_plot}.svg', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
# plt.savefig(f'{filename_box_plot}.png', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)

#########

plt.show()
print('file saved at :', filename_box_plot)

In [ ]:
print('Background type:\t',background_type)
print('Total Time: \t\t',str(datetime.timedelta(seconds=np.sum(data_frame_for_plotting.time_total))))

# END